<a href="https://colab.research.google.com/github/Roberto-he/TFM_RHG/blob/main/TFM_FIRE_RHG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OBJETIVO 

Con el bjetivo de este proyecto es buscar los incendios reales que se producen en Londres

El dataset utilizdo ha sido publicado por la la brigada de incendios de londres (LFB)


Las tareas realizadas sobre el data set son:
- Limpiar el dataset y tratar los valores "NaN"
- Buscar datos perdidos
- Balanceo de datos 
- Analizar los datos analiticamente y con graficas
- Buscar un algoritmo para predecir cuando hay un incendio o cuando es una falsa alarma.


#LIBRERIAS

In [ ]:
# carga de datos
import pandas as pd
import io
from google.colab import files
# manipulación y visualización
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


#CARGA DE DATASET DE GRAN TAMANYO EN GOOGLE DRIVE

##CONVERSION DEL DATASET A CSV

In [ ]:
def upload_files ():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')))
    return df

In [ ]:
def upload_files_xls ():
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_excel(io.StringIO(uploaded[fn].decode('utf-8')))
    return df

In [ ]:
df1 = upload_files_xls()
df1.head()

NameError: ignored

In [ ]:
df2 = upload_files_xls()
df2.head()

In [ ]:
df3 = upload_files_xls()
df3.head()

##UNION DE LOS DATASETS

In [ ]:
#Unir las 3 bases de datos
frames = [df1, df2, df3]

df_excel = pd.concat(frames)

In [ ]:
#Pasar a CSV
df_excel.to_csv('LFB incident 2009-2021.csv', index=False)

In [ ]:
#Exportar CSV

from google.colab import files

with open('LFB incident 2009-2021.csv', 'w') as f:
  df.to_csv(f,  index = False)
  

files.download('LFB incident 2009-2021.csv')

df.head()

In [ ]:
df=pd.read_csv("LFB incident 2009-2021.csv")
df

## UNZIP
Al ser un archivo muy pesado lo subo a ggole colab comprimido. Ya en colab lo descomprimo

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!unrar x  '/gdrive/My Drive/TFM-Roberto_Hernandez/LFB_incident_2009-2021.rar'  "/gdrive/My Drive/TFM-Roberto_Hernandez/"


#CARGA DATASETS

##DATASET INCENDIOS

In [ ]:
#Url: https://data.gov.uk/dataset/e6b9d1c6-7c34-42ca-a100-cd62164ea76c/london-fire-brigade-incident-records
llamadas = pd.read_csv('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/LFB incident 2009-2021.csv')
llamadas.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,UPRN,USRN,Latitude,Longitude,PumpCount,PumpHoursRoundUp,Notional Cost (£)
0,235138081,2009-01-01,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,In street close to gazetteer location,SW11 4LB,SW11,E09000032,WANDSWORTH,Wandsworth,E05000620,Queenstown,Queenstown,528652.0,176830.0,528650,176850,London,Battersea,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1091,2009-01-01,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,Open land/water - nearest gazetteer location,SE1 7SG,SE1,E09000022,LAMBETH,Lambeth,E05000416,Bishop's,Bishop's,530485.0,179007.0,530450,179050,London,Lambeth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009-01-01,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,In street outside gazetteer location,N9 9EL,N9,E09000010,ENFIELD,Enfield,E05000201,Haselbury,Haselbury,533773.0,194492.0,533750,194450,London,Edmonton,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3091,2009-01-01,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),On land associated with building,UB10 0DG,UB10,E09000017,HILLINGDON,Hillingdon,E05000332,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,London,Hillingdon,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5091,2009-01-01,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,In street outside gazetteer location,N7 8HG,N7,E09000019,ISLINGTON,Islington,E05000375,Holloway,Holloway,531058.0,185307.0,531050,185350,London,Holloway,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Vemos la aparte de abajo de nuestro dataset
llamadas.tail()


,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending,UPRN,USRN,Latitude,Longitude,PumpCount,PumpHoursRoundUp,Notional Cost (£)
1333065,050590-30042021,2021-04-30,2021,21:42:23,21,Special Service,Special Service,Lift Release,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,NaN,NW6,E09000007,CAMDEN,Camden,E05000144,SWISS COTTAGE,SWISS COTTAGE,NaN,NaN,525650,184050,London,West Hampstead,NaN,NaN,NaN,NaN,1.0,1.0,0.000000e+00,20400412.0,NaN,NaN,1.0,1.0,352.0
1333066,050431-30042021,2021-04-30,2021,16:20:38,16,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,Correct incident location,SE8 5DD,SE8,E09000023,LEWISHAM,Lewisham,E05000443,EVELYN,EVELYN,536272.0,178387.0,536250,178350,London,Deptford,402.0,Old Kent Road,NaN,NaN,1.0,1.0,1.002533e+10,22004367.0,51.488022,-0.038628,1.0,1.0,352.0
1333067,050653-30042021,2021-04-30,2021,23:53:36,23,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,Nearby address - street not listed in gazetteer,E10 7AG,E10,E09000031,WALTHAM FOREST,Waltham Forest,E05000603,LEA BRIDGE,LEA BRIDGE,537667.0,187977.0,537650,187950,London,Leyton,244.0,Leyton,NaN,NaN,1.0,1.0,1.000236e+11,22832450.0,51.573868,-0.014799,1.0,1.0,352.0
1333068,050428-30042021,2021-04-30,2021,16:14:59,16,Fire,Secondary Fire,NaN,Outdoor Structure,Common external bin storage area,In street close to gazetteer location,SE13 7LX,SE13,E09000011,GREENWICH,Greenwich,E05000222,GREENWICH WEST,GREENWICH WEST,537885.0,176420.0,537850,176450,London,Greenwich,452.0,Greenwich,NaN,NaN,1.0,1.0,1.000210e+11,20801074.0,51.469960,-0.016181,1.0,1.0,352.0
1333069,050249-30042021,2021-04-30,2021,09:44:37,9,Special Service,Special Service,Animal assistance incidents,Outdoor,Animal harm outdoors,On land associated with building,DA14 5EP,DA14,E09000004,BEXLEY,Bexley,E05011229,ST. MARY'S & ST. JAMES,ST. MARY'S & ST. JAMES,548763.0,172019.0,548750,172050,London,Sidcup,443.0,Bexley,NaN,NaN,1.0,1.0,1.002330e+10,20101032.0,51.427664,0.138471,2.0,1.0,352.0


In [ ]:
# tamno del dataset
llamadas.shape

(1333070, 38)

In [ ]:
# Informacion asociada al dataset
llamadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333070 entries, 0 to 1333069
Data columns (total 38 columns):
 #   Column                                  Non-Null Count    Dtype  
---  ------                                  --------------    -----  
 0   IncidentNumber                          1333070 non-null  object 
 1   DateOfCall                              1333070 non-null  object 
 2   CalYear                                 1333070 non-null  int64  
 3   TimeOfCall                              1333070 non-null  object 
 4   HourOfCall                              1333070 non-null  int64  
 5   IncidentGroup                           1333069 non-null  object 
 6   StopCodeDescription                     1333069 non-null  object 
 7   SpecialServiceType                      408181 non-null   object 
 8   PropertyCategory                        1333069 non-null  object 
 9   PropertyType                            1333069 non-null  object 
 10  AddressQualifier              

In [ ]:
null_columns=llamadas.columns[llamadas.isnull().any()]
llamadas[null_columns].isnull().sum()

IncidentGroup                                   1
StopCodeDescription                             1
SpecialServiceType                         924889
PropertyCategory                                1
PropertyType                                    1
Postcode_full                              629843
Easting_m                                  629843
Northing_m                                 629843
FirstPumpArriving_AttendanceTime           234730
FirstPumpArriving_DeployedFromStation      234740
SecondPumpArriving_AttendanceTime         1162974
SecondPumpArriving_DeployedFromStation    1162978
NumStationsWithPumpsAttending                7686
NumPumpsAttending                            7686
UPRN                                       889041
USRN                                       889041
Latitude                                  1116267
Longitude                                 1116267
PumpCount                                  891065
PumpHoursRoundUp                           891205


###ANALISIS DE DATOS

In [ ]:
def descripcionDatosDataset(datos):
    print("Cantidad de filas:", datos.shape[0])
    print("Cantidad de columnas:", datos.shape[1])
    print('-'*100)
    for columna in datos.columns:
        valoresDescripcion = ''
        tipo = ''
        if datos[columna].dtype == 'float64' or datos[columna].dtype == 'int64':
            tipo = 'numérico'
            valoresDescripcion = datos[columna].agg(['min', 'max', 'mean', 'std', 'median'])
        else:
            tipo = 'nominal' #categórico | string | no-numérico
            valoresDescripcion = {'valoresPresentes' : datos[columna].unique(), 
                                  'cantidadNulos' : datos[columna].isna().sum()}
                                 #'conteoValores' : pd.value_counts(datos[columna])}
        print('Columna: ' + columna)
        print('Tipo de datos: ' + tipo)
        print('Descripción de valores:')
        if tipo == 'numérico':
            print(valoresDescripcion)
        else:
            print('-- Valores presentes (10 primeros): ' + str(valoresDescripcion['valoresPresentes'][:10]))
            pctNulos = (valoresDescripcion['cantidadNulos'] / datos.shape[0]) * 100
            print('-- Cantidad de nulos: ' + str(valoresDescripcion['cantidadNulos']) + ' = ' + "{0:.2f}".format(pctNulos) + '%')
        print('-'*100)

In [ ]:
descripcionDatosDataset(llamadas)

Cantidad de filas: 1333070
Cantidad de columnas: 38
----------------------------------------------------------------------------------------------------
Columna: IncidentNumber
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): [235138081 1091 2091 3091 5091 6091 8091 9091 10091 11091]
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: DateOfCall
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['2009-01-01' '2009-01-02' '2009-01-03' '2009-01-04' '2009-01-05'
 '2009-01-06' '2009-01-07' '2009-01-08' '2009-01-09' '2009-01-10']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: CalYear
Tipo de datos: numérico
Descripción de valores:
min       2009.000000
max       2021.000000
mean      2014.398667
std          3.632486
median    2014.000000
Name

###LIMPIEZA

In [ ]:
#Borramos los atributos posteriores a la llamda que no aportan nada en nuestro estudio
col_eliminar_llamadas=["IncidentStationGround","FirstPumpArriving_AttendanceTime","FirstPumpArriving_DeployedFromStation","SecondPumpArriving_AttendanceTime","SecondPumpArriving_DeployedFromStation","NumStationsWithPumpsAttending","NumPumpsAttending","PumpHoursRoundUp","PumpCount"]
llamadas.drop(col_eliminar_llamadas, inplace=True, axis=1)
llamadas

,IncidentNumber,DateOfCall,CalYear,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,IncGeo_BoroughCode,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,FRS,NumPumpsAttending,UPRN,USRN,Latitude,Longitude,PumpCount,PumpHoursRoundUp,Notional Cost (£)
0,235138081,2009-01-01,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,In street close to gazetteer location,SW11 4LB,SW11,E09000032,WANDSWORTH,Wandsworth,E05000620,Queenstown,Queenstown,528652.0,176830.0,528650,176850,London,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1091,2009-01-01,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,Open land/water - nearest gazetteer location,SE1 7SG,SE1,E09000022,LAMBETH,Lambeth,E05000416,Bishop's,Bishop's,530485.0,179007.0,530450,179050,London,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009-01-01,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,In street outside gazetteer location,N9 9EL,N9,E09000010,ENFIELD,Enfield,E05000201,Haselbury,Haselbury,533773.0,194492.0,533750,194450,London,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3091,2009-01-01,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),On land associated with building,UB10 0DG,UB10,E09000017,HILLINGDON,Hillingdon,E05000332,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,London,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5091,2009-01-01,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,In street outside gazetteer location,N7 8HG,N7,E09000019,ISLINGTON,Islington,E05000375,Holloway,Holloway,531058.0,185307.0,531050,185350,London,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333065,050590-30042021,2021-04-30,2021,21:42:23,21,Special Service,Special Service,Lift Release,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,NaN,NW6,E09000007,CAMDEN,Camden,E05000144,SWISS COTTAGE,SWISS COTTAGE,NaN,NaN,525650,184050,London,1.0,0.000000e+00,20400412.0,NaN,NaN,1.0,1.0,352.0
1333066,050431-30042021,2021-04-30,2021,16:20:38,16,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,Correct incident location,SE8 5DD,SE8,E09000023,LEWISHAM,Lewisham,E05000443,EVELYN,EVELYN,536272.0,178387.0,536250,178350,London,1.0,1.002533e+10,22004367.0,51.488022,-0.038628,1.0,1.0,352.0
1333067,050653-30042021,2021-04-30,2021,23:53:36,23,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,Nearby address - street not listed in gazetteer,E10 7AG,E10,E09000031,WALTHAM FOREST,Waltham Forest,E05000603,LEA BRIDGE,LEA BRIDGE,537667.0,187977.0,537650,187950,London,1.0,1.000236e+11,22832450.0,51.573868,-0.014799,1.0,1.0,352.0
1333068,050428-30042021,2021-04-30,2021,16:14:59,16,Fire,Secondary Fire,NaN,Outdoor Structure,Common external bin storage area,In street close to gazetteer location,SE13 7LX,SE13,E09000011,GREENWICH,Greenwich,E05000222,GREENWICH WEST,GREENWICH WEST,537885.0,176420.0,537850,176450,London,1.0,1.000210e+11,20801074.0,51.469960,-0.016181,1.0,1.0,352.0


In [ ]:
#Renombramos las columnas para poder hacer las uniones entre los distintos datasets
llamadas=llamadas.rename(columns={"IncGeo_BoroughCode":"Code", "CalYear":"Year"})
llamadas

,IncidentNumber,DateOfCall,Year,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,Code,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,FRS,NumPumpsAttending,UPRN,USRN,Latitude,Longitude,PumpCount,PumpHoursRoundUp,Notional Cost (£)
0,235138081,2009-01-01,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,In street close to gazetteer location,SW11 4LB,SW11,E09000032,WANDSWORTH,Wandsworth,E05000620,Queenstown,Queenstown,528652.0,176830.0,528650,176850,London,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1091,2009-01-01,2009,00:00:46,0,Special Service,Special Service,Assist other agencies,Outdoor,Lake/pond/reservoir,Open land/water - nearest gazetteer location,SE1 7SG,SE1,E09000022,LAMBETH,Lambeth,E05000416,Bishop's,Bishop's,530485.0,179007.0,530450,179050,London,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2091,2009-01-01,2009,00:03:00,0,Fire,Secondary Fire,NaN,Outdoor,Road surface/pavement,In street outside gazetteer location,N9 9EL,N9,E09000010,ENFIELD,Enfield,E05000201,Haselbury,Haselbury,533773.0,194492.0,533750,194450,London,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3091,2009-01-01,2009,00:04:27,0,Fire,Secondary Fire,NaN,Outdoor,Domestic garden (vegetation not equipment),On land associated with building,UB10 0DG,UB10,E09000017,HILLINGDON,Hillingdon,E05000332,Hillingdon East,Hillingdon East,507738.0,182805.0,507750,182850,London,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5091,2009-01-01,2009,00:05:39,0,Fire,Secondary Fire,NaN,Outdoor,Cycle path/public footpath/bridleway,In street outside gazetteer location,N7 8HG,N7,E09000019,ISLINGTON,Islington,E05000375,Holloway,Holloway,531058.0,185307.0,531050,185350,London,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333065,050590-30042021,2021-04-30,2021,21:42:23,21,Special Service,Special Service,Lift Release,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,NaN,NW6,E09000007,CAMDEN,Camden,E05000144,SWISS COTTAGE,SWISS COTTAGE,NaN,NaN,525650,184050,London,1.0,0.000000e+00,20400412.0,NaN,NaN,1.0,1.0,352.0
1333066,050431-30042021,2021-04-30,2021,16:20:38,16,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,Correct incident location,SE8 5DD,SE8,E09000023,LEWISHAM,Lewisham,E05000443,EVELYN,EVELYN,536272.0,178387.0,536250,178350,London,1.0,1.002533e+10,22004367.0,51.488022,-0.038628,1.0,1.0,352.0
1333067,050653-30042021,2021-04-30,2021,23:53:36,23,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,Nearby address - street not listed in gazetteer,E10 7AG,E10,E09000031,WALTHAM FOREST,Waltham Forest,E05000603,LEA BRIDGE,LEA BRIDGE,537667.0,187977.0,537650,187950,London,1.0,1.000236e+11,22832450.0,51.573868,-0.014799,1.0,1.0,352.0
1333068,050428-30042021,2021-04-30,2021,16:14:59,16,Fire,Secondary Fire,NaN,Outdoor Structure,Common external bin storage area,In street close to gazetteer location,SE13 7LX,SE13,E09000011,GREENWICH,Greenwich,E05000222,GREENWICH WEST,GREENWICH WEST,537885.0,176420.0,537850,176450,London,1.0,1.000210e+11,20801074.0,51.469960,-0.016181,1.0,1.0,352.0


In [ ]:
llamadas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1333070 entries, 0 to 1333069
Data columns (total 32 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   IncidentNumber       1333070 non-null  object 
 1   DateOfCall           1333070 non-null  object 
 2   Year                 1333070 non-null  int64  
 3   TimeOfCall           1333070 non-null  object 
 4   HourOfCall           1333070 non-null  int64  
 5   IncidentGroup        1333069 non-null  object 
 6   StopCodeDescription  1333069 non-null  object 
 7   SpecialServiceType   408181 non-null   object 
 8   PropertyCategory     1333069 non-null  object 
 9   PropertyType         1333069 non-null  object 
 10  AddressQualifier     1333070 non-null  object 
 11  Postcode_full        703227 non-null   object 
 12  Postcode_district    1333070 non-null  object 
 13  Code                 1333070 non-null  object 
 14  IncGeo_BoroughName   1333070 non-null  object 
 15

## DATASET PRECIO VIVIENDA

In [ ]:
viviendas = pd.read_excel('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/ratio-house-price-earnings-residence-based.xlsx')
viviendas.head()


,New Code,Old Code,Area,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E09000001,00AA,City of London,6.29,:,:,:,:,:,:,9.52,:,8.66,:,:,:,:,14.83,:,16.55,:,:
2,E09000002,00AB,Barking and Dagenham,4.68,6.26,7.18,7.15,6.55,7.05,7.46,5.71,6.3,6.56,6.38,6.28,7.54,8.24,9.89,10,10.40,10.75,9.63
3,E09000003,00AC,Barnet,7.42,8.49,8.64,8.46,9.12,9.83,9.16,8.47,9.7,10.06,10.05,10.16,12.14,13.3,14.05,13.7,14.45,14.19,15.93
4,E09000004,00AD,Bexley,5.56,6.29,6.57,6.73,6.79,7.19,6.97,6.29,6.72,6.77,6.53,6.68,7.69,8.47,9.44,10.01,10.04,9.61,9.28


###ANALISIS DE DATOS

In [ ]:
descripcionDatosDataset(viviendas)

Cantidad de filas: 46
Cantidad de columnas: 22
----------------------------------------------------------------------------------------------------
Columna: New Code
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): [nan 'E09000001' 'E09000002' 'E09000003' 'E09000004' 'E09000005'
 'E09000006' 'E09000007' 'E09000008' 'E09000009']
-- Cantidad de nulos: 3 = 6.52%
----------------------------------------------------------------------------------------------------
Columna: Old Code
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): [nan '00AA' '00AB' '00AC' '00AD' '00AE' '00AF' '00AG' '00AH' '00AJ']
-- Cantidad de nulos: 3 = 6.52%
----------------------------------------------------------------------------------------------------
Columna: Area
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): [nan 'City of London' 'Barking and Dagenham' 'Barnet' 'Bexley' 'Brent'
 'Bromley' 'Camden' 'Croydon' 'E

TypeError: ignored

###TRANSFORMACION

Realizamos las transformaciones necesarias para poder unirlo

In [ ]:
#Transformacion de dataset para poder unirlo al principal
viv=pd.DataFrame(columns = ['Code', 'Precio', "Year"])
viviendas_tr=pd.DataFrame(columns = ['Code', 'Precio',"Year"])
for i in range(1,46):
  viv["Precio"]=viviendas.iloc[i,3:22]
  viv["Code"]=viviendas.iloc[i,0]
  viv["Year"]=viviendas.columns[3:22]
  viviendas_tr=pd.concat([viviendas_tr,viv])
viviendas_tr

,Code,Precio,Year
2002,E09000001,6.29,2002
2003,E09000001,:,2003
2004,E09000001,:,2004
2005,E09000001,:,2005
2006,E09000001,:,2006
...,...,...,...
2016,E92000001,7.72,2016
2017,E92000001,7.91,2017
2018,E92000001,8.04,2018
2019,E92000001,7.88,2019


##DATASET DENSIDAD POBLACION

In [ ]:
densidad = pd.read_csv('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/housing-density-borough.csv')
densidad.head()

,Code,Name,Year,Source,Population,Inland_Area _Hectares,Total_Area_Hectares,Population_per_hectare,Square_Kilometres,Population_per_square_kilometre
0,E09000001,City of London,1999,ONS MYE,6581,290.4,314.9,22.7,2.9,2266.2
1,E09000001,City of London,2000,ONS MYE,7014,290.4,314.9,24.2,2.9,2415.3
2,E09000001,City of London,2001,ONS MYE,7359,290.4,314.9,25.3,2.9,2534.1
3,E09000001,City of London,2002,ONS MYE,7280,290.4,314.9,25.1,2.9,2506.9
4,E09000001,City of London,2003,ONS MYE,7115,290.4,314.9,24.5,2.9,2450.1


###ANALISIS DE DATOS

In [ ]:
descripcionDatosDataset(densidad)

Cantidad de filas: 1872
Cantidad de columnas: 10
----------------------------------------------------------------------------------------------------
Columna: Code
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['E09000001' 'E09000002' 'E09000003' 'E09000004' 'E09000005' 'E09000006'
 'E09000007' 'E09000008' 'E09000009' 'E09000010']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Name
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['City of London' 'Barking and Dagenham' 'Barnet' 'Bexley' 'Brent'
 'Bromley' 'Camden' 'Croydon' 'Ealing' 'Enfield']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Year
Tipo de datos: numérico
Descripción de valores:
min       1999.000000
max       2050.000000
mean      2024.500000
std         15.012341


###LIMPIEZA

In [ ]:
col_eliminar_densidad=["Name","Source","Inland_Area _Hectares", "Total_Area_Hectares", "Population_per_hectare", "Square_Kilometres"]
densidad.drop(col_eliminar_densidad, inplace=True, axis=1)
densidad

,Code,Year,Population,Population_per_square_kilometre
0,E09000001,1999,6581,2266.2
1,E09000001,2000,7014,2415.3
2,E09000001,2001,7359,2534.1
3,E09000001,2002,7280,2506.9
4,E09000001,2003,7115,2450.1
...,...,...,...,...
1867,E13000002,2046,6573194,5240.8
1868,E13000002,2047,6598789,5261.2
1869,E13000002,2048,6622921,5280.4
1870,E13000002,2049,6647527,5300.1


In [ ]:
#Renombramos las columnas para poder hacer las uniones entre los distintos datasets
densidad=densidad.rename(columns={"Population":"Poblacion", "Population_per_square_kilometre":"Poblacion_m2"})
densidad

,Code,Year,Poblacion,Poblacion_m2
0,E09000001,1999,6581,2266.2
1,E09000001,2000,7014,2415.3
2,E09000001,2001,7359,2534.1
3,E09000001,2002,7280,2506.9
4,E09000001,2003,7115,2450.1
...,...,...,...,...
1867,E13000002,2046,6573194,5240.8
1868,E13000002,2047,6598789,5261.2
1869,E13000002,2048,6622921,5280.4
1870,E13000002,2049,6647527,5300.1


##DATASET CRIMENES

In [ ]:
crimenes = pd.read_csv('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/crime-rates.csv')
crimenes.head()

,Code,Borough,Year,Offences,Rate,Number_of_offences
0,E09000002,Barking and Dagenham,1999-00,All recorded offences,120.5,"19,567"
1,E09000003,Barnet,1999-00,All recorded offences,98.0,"30,708"
2,E09000004,Bexley,1999-00,All recorded offences,95.1,"20,680"
3,E09000005,Brent,1999-00,All recorded offences,127.7,"33,253"
4,E09000006,Bromley,1999-00,All recorded offences,89.8,"26,474"


###ANALISIS DE DATOS

In [ ]:
descripcionDatosDataset(crimenes)

Cantidad de filas: 6660
Cantidad de columnas: 6
----------------------------------------------------------------------------------------------------
Columna: Code
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['E09000002' 'E09000003' 'E09000004' 'E09000005' 'E09000006' 'E09000007'
 'E09000008' 'E09000009' 'E09000010' 'E09000011']
-- Cantidad de nulos: 180 = 2.70%
----------------------------------------------------------------------------------------------------
Columna: Borough
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['Barking and Dagenham' 'Barnet' 'Bexley' 'Brent' 'Bromley' 'Camden'
 'Croydon' 'Ealing' 'Enfield' 'Greenwich']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Year
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['1999-00' '2000-01' '2001-02' '2002-03' '2003-04' '20

###LIMPIEZA

In [ ]:
col_eliminar_crimenes=["Borough", "Offences", "Rate"]
crimenes.drop(col_eliminar_crimenes, inplace=True, axis=1)
crimenes

,Code,Year,Number_of_offences
0,E09000002,1999-00,"19,567"
1,E09000003,1999-00,"30,708"
2,E09000004,1999-00,"20,680"
3,E09000005,1999-00,"33,253"
4,E09000006,1999-00,"26,474"
...,...,...,...
6655,NaN,2016-17,"1,081"
6656,E13000001,2016-17,"6,041"
6657,E13000002,2016-17,"6,637"
6658,E12000007,2016-17,"13,759"


In [ ]:
#Cambio el formato de la fercha de los crimenes solo no interesa el anyo
crimenes["Year"]=crimenes["Year"].map(lambda x: str(x)[:-3])
crimenes["Year"]

0       1999
1       1999
2       1999
3       1999
4       1999
        ... 
6655    2016
6656    2016
6657    2016
6658    2016
6659    2016
Name: Year, Length: 6660, dtype: object

##DATASET PRECIO VIVIENDA

In [ ]:
p_vivienda = pd.read_csv('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/land-registry-house-prices-borough.csv')
p_vivienda.head()

,Code,Area,Year,Measure,Value
0,E09000001,City of London,Year ending Dec 1995,Median,"105,000"
1,E09000002,Barking and Dagenham,Year ending Dec 1995,Median,"49,000"
2,E09000003,Barnet,Year ending Dec 1995,Median,"85,125"
3,E09000004,Bexley,Year ending Dec 1995,Median,"62,000"
4,E09000005,Brent,Year ending Dec 1995,Median,"68,000"


###ANALISIS DE DATOS

In [ ]:
descripcionDatosDataset(p_vivienda)

Cantidad de filas: 12015
Cantidad de columnas: 5
----------------------------------------------------------------------------------------------------
Columna: Code
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['E09000001' 'E09000002' 'E09000003' 'E09000004' 'E09000005' 'E09000006'
 'E09000007' 'E09000008' 'E09000009' 'E09000010']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Area
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['City of London' 'Barking and Dagenham' 'Barnet' 'Bexley' 'Brent'
 'Bromley' 'Camden' 'Croydon' 'Ealing' 'Enfield']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Year
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['Year ending Dec 1995' 'Year ending Mar 1996' 'Year 

###LIMPIEZA

In [ ]:
#Nos quedamos con las ventas
p_vivienda=p_vivienda.loc[p_vivienda["Measure"]=="Sales"]
p_vivienda

,Code,Area,Year,Measure,Value
8010,E09000001,City of London,Year ending Dec 1995,Sales,165
8011,E09000002,Barking and Dagenham,Year ending Dec 1995,Sales,"1,488"
8012,E09000003,Barnet,Year ending Dec 1995,Sales,"4,758"
8013,E09000004,Bexley,Year ending Dec 1995,Sales,"3,430"
8014,E09000005,Brent,Year ending Dec 1995,Sales,"3,473"
...,...,...,...,...,...
12010,E12000008,South East,Year ending Dec 2017,Sales,"148,025"
12011,E12000009,South West,Year ending Dec 2017,Sales,"102,292"
12012,E92000001,England,Year ending Dec 2017,Sales,"837,501"
12013,W92000076,Wales,Year ending Dec 2017,Sales,"46,828"


In [ ]:
col_eliminar_vivienda=["Area", "Measure"]
p_vivienda.drop(col_eliminar_vivienda, inplace=True, axis=1)
p_vivienda

,Code,Year,Value
8010,E09000001,Year ending Dec 1995,165
8011,E09000002,Year ending Dec 1995,"1,488"
8012,E09000003,Year ending Dec 1995,"4,758"
8013,E09000004,Year ending Dec 1995,"3,430"
8014,E09000005,Year ending Dec 1995,"3,473"
...,...,...,...
12010,E12000008,Year ending Dec 2017,"148,025"
12011,E12000009,Year ending Dec 2017,"102,292"
12012,E92000001,Year ending Dec 2017,"837,501"
12013,W92000076,Year ending Dec 2017,"46,828"


In [ ]:
#Transformamos la columna anyo para poder unir el dataset al resto
p_vivienda["Year"]=p_vivienda["Year"].map(lambda x: str(x)[16:])
p_vivienda

,Code,Year,Value
8010,E09000001,1995,165
8011,E09000002,1995,"1,488"
8012,E09000003,1995,"4,758"
8013,E09000004,1995,"3,430"
8014,E09000005,1995,"3,473"
...,...,...,...
12010,E12000008,2017,"148,025"
12011,E12000009,2017,"102,292"
12012,E92000001,2017,"837,501"
12013,W92000076,2017,"46,828"


##DATASET CONSUMO ENERGETICO

In [ ]:
consumo = pd.read_csv('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/total-final-energy-consumption.csv')
consumo.head()

,Code,Area,Year,Fuel,Sector,Measurement,Value
0,E09000002,Barking and Dagenham,2005,Coal,Industrial & Commercial,Ktoe,0
1,E09000003,Barnet,2005,Coal,Industrial & Commercial,Ktoe,0
2,E09000004,Bexley,2005,Coal,Industrial & Commercial,Ktoe,0.008194822
3,E09000005,Brent,2005,Coal,Industrial & Commercial,Ktoe,0
4,E09000006,Bromley,2005,Coal,Industrial & Commercial,Ktoe,0.002010728


###ANALISIS DE DATOS

In [ ]:
descripcionDatosDataset(consumo)

Cantidad de filas: 23782
Cantidad de columnas: 7
----------------------------------------------------------------------------------------------------
Columna: Code
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['E09000002' 'E09000003' 'E09000004' 'E09000005' 'E09000006' 'E09000007'
 'E09000001' 'E09000008' 'E09000009' 'E09000010']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Area
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['Barking and Dagenham' 'Barnet' 'Bexley' 'Brent' 'Bromley' 'Camden'
 'City of London' 'Croydon' 'Ealing' 'Enfield']
-- Cantidad de nulos: 0 = 0.00%
----------------------------------------------------------------------------------------------------
Columna: Year
Tipo de datos: numérico
Descripción de valores:
min       2005.000000
max       2015.000000
mean      2010.000000
std          3.162344


###LIMPIEZA

In [ ]:
#Borramos las columnas que no necesitamos
consumo=consumo.loc[consumo["Measurement"]=="GWh"]
col_eliminar_consumo=["Area", "Measurement"]
consumo.drop(col_eliminar_consumo, inplace=True, axis=1)
consumo=consumo.astype("string")
#consumo=consumo(["Code","Year", "Fuel"], dtype="string")
consumo

,Code,Year,Fuel,Sector,Value
11891,E09000002,2005,Coal,Industrial & Commercial,0
11892,E09000003,2005,Coal,Industrial & Commercial,0
11893,E09000004,2005,Coal,Industrial & Commercial,0.095305777
11894,E09000005,2005,Coal,Industrial & Commercial,0
11895,E09000006,2005,Coal,Industrial & Commercial,0.023384762
...,...,...,...,...,...
23777,N92000002,2015,Consuming Sector,Transport,0
23778,W92000004,2015,Consuming Sector,Transport,0
23779,S92000003,2015,Consuming Sector,Transport,0
23780,E92000001,2015,Consuming Sector,Transport,0


In [ ]:
#Creamos un dataset por cada tipo de energia consumida
consumo_coal=consumo.loc[consumo["Fuel"]=="Coal"]
consumo_coal=consumo_coal.rename(columns={"Value": "Coal"})
consumo_coal.drop("Fuel", inplace=True, axis=1)

consumo_manuf=consumo.loc[consumo["Fuel"]=="Manufactured fuels"]
consumo_manuf=consumo_manuf.rename(columns={"Value": "Manufacturer"})
consumo_manuf.drop("Fuel", inplace=True, axis=1)

consumo_petroleum=consumo.loc[consumo["Fuel"]=="Petroleum products"]
consumo_petroleum=consumo_petroleum.rename(columns={"Value": "Petroleum"})
consumo_petroleum.drop("Fuel", inplace=True, axis=1)

consumo_gas=consumo.loc[consumo["Fuel"]=="Gas"]
consumo_gas=consumo_gas.rename(columns={"Value": "Gas"})
consumo_gas.drop("Fuel", inplace=True, axis=1)

consumo_elect=consumo.loc[consumo["Fuel"]=="Electricity"]
consumo_elect=consumo_elect.rename(columns={"Value": "Electricity"})
consumo_elect.drop("Fuel", inplace=True, axis=1)

consumo_bio=consumo.loc[consumo["Fuel"]=="Bioenergy & wastes"]
consumo_bio=consumo_bio.rename(columns={"Value": "Bionergy"})
consumo_bio.drop("Fuel", inplace=True, axis=1)

###UNION DATASETS

In [ ]:
from functools import reduce
dfs = [consumo_coal, consumo_manuf, consumo_petroleum, consumo_gas,consumo_elect,consumo_bio]
consumo_total = reduce(lambda left,right: pd.merge(left,right, how="outer"), dfs)
consumo_total

,Code,Year,Sector,Coal,Manufacturer,Petroleum,Gas,Electricity,Bionergy
0,E09000002,2005,Industrial & Commercial,0,<NA>,164.5881512,325.524111,479.039394,<NA>
1,E09000003,2005,Industrial & Commercial,0,<NA>,64.88691195,808.225172,554.666592,<NA>
2,E09000004,2005,Industrial & Commercial,0.095305777,<NA>,285.4180887,895.424066,444.617436,<NA>
3,E09000005,2005,Industrial & Commercial,0,<NA>,117.0203914,639.532412,757.595565,<NA>
4,E09000006,2005,Industrial & Commercial,0.023384762,<NA>,91.83366408,567.762634,477.964961,<NA>
...,...,...,...,...,...,...,...,...,...
3097,N92000002,2015,Road,<NA>,<NA>,5177.430218,<NA>,<NA>,<NA>
3098,W92000004,2015,Road,<NA>,<NA>,10699.84383,<NA>,<NA>,<NA>
3099,S92000003,2015,Road,<NA>,<NA>,21057.84046,<NA>,<NA>,<NA>
3100,E92000001,2015,Road,<NA>,<NA>,77124.29437,<NA>,<NA>,<NA>


##DATASET FESTIVOS

In [ ]:
festivos = pd.read_excel('/gdrive/My Drive/TFM-Roberto_Hernandez/datasets/Bank-holidays.xlsx')
festivos.head()

,Date,Holiday-Event,Holiday-code
0,1995-01-01,New Year's Day,F
1,1995-01-02,New Year's Day-moved,M
2,1995-04-13,Previous Good Friday,P
3,1995-04-14,Good Friday,F
4,1995-04-17,Easter Monday,F


###ANALISIS DE DATOS

In [ ]:
descripcionDatosDataset(festivos)

Cantidad de filas: 141
Cantidad de columnas: 3
----------------------------------------------------------------------------------------------------
Columna: Date
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ['1995-01-01T00:00:00.000000000' '1995-01-02T00:00:00.000000000'
 '1995-04-13T00:00:00.000000000' '1995-04-14T00:00:00.000000000'
 '1995-04-17T00:00:00.000000000' '1995-05-08T00:00:00.000000000'
 '1995-05-29T00:00:00.000000000' '1995-08-28T00:00:00.000000000'
 '1995-05-05T00:00:00.000000000' '1995-12-25T00:00:00.000000000']
-- Cantidad de nulos: 12 = 8.51%
----------------------------------------------------------------------------------------------------
Columna: Holiday-Event
Tipo de datos: nominal
Descripción de valores:
-- Valores presentes (10 primeros): ["New Year's Day" "New Year's Day-moved" 'Previous Good Friday'
 'Good Friday' 'Easter Monday' 'Early May bank holiday'
 'Late May bank holiday' 'August Bank Holidays' 'Guy Fawkes Day'
 'Ch

###LIMPIEZA

#DATASET FINAL

Unimos todos los dataset, usaremos el codigo del barrio como referencia

###UNION DATASETS

In [ ]:
#Unimos los dataset por el codigo y el anyo
dfs = [llamadas.astype("string"), viviendas_tr.astype("string")]
consumo_total = reduce(lambda left,right: pd.merge(left,right, how="outer"), dfs)
consumo_total

,IncidentNumber,DateOfCall,Year,TimeOfCall,HourOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,Postcode_district,Code,IncGeo_BoroughName,ProperCase,IncGeo_WardCode,IncGeo_WardName,IncGeo_WardNameNew,Easting_m,Northing_m,Easting_rounded,Northing_rounded,FRS,NumPumpsAttending,UPRN,USRN,Latitude,Longitude,PumpCount,PumpHoursRoundUp,Notional Cost (£),Precio
0,235138081,2009-01-01,2009,00:00:37,0,Special Service,Special Service,RTC,Road Vehicle,Car,In street close to gazetteer location,SW11 4LB,SW11,E09000032,WANDSWORTH,Wandsworth,E05000620,Queenstown,Queenstown,528652.0,176830.0,528650,176850,London,2.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9.14
1,155091,2009-01-01,2009,03:40:14,3,False Alarm,AFA,<NA>,Dwelling,Self contained Sheltered Housing,Correct incident location,<NA>,SW11,E09000032,WANDSWORTH,Wandsworth,E05000622,St. Mary's Park,St. Mary's Park,<NA>,<NA>,526850,176750,London,2.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9.14
2,232091,2009-01-01,2009,06:58:52,6,Fire,Secondary Fire,<NA>,Outdoor Structure,Small refuse/rubbish container,In street outside gazetteer location,SW15 4DG,SW15,E09000032,WANDSWORTH,Wandsworth,E05000621,Roehampton and Putney Heath,Roehampton and Putney Heath,522250.0,173750.0,522250,173750,London,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9.14
3,248091,2009-01-01,2009,07:55:23,7,Special Service,Special Service,Effecting entry/exit,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Correct incident location,<NA>,SW11,E09000032,WANDSWORTH,Wandsworth,E05000623,Shaftesbury,Shaftesbury,<NA>,<NA>,528350,175850,London,1.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9.14
4,315091,2009-01-01,2009,10:47:07,10,False Alarm,AFA,<NA>,Other Residential,Nursing/Care Home/Hospice,Correct incident location,SW15 2UQ,SW15,E09000032,WANDSWORTH,Wandsworth,E05000613,East Putney,East Putney,524549.0,174454.0,524550,174450,London,3.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333524,<NA>,<NA>,2016,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,E92000001,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7.72
1333525,<NA>,<NA>,2017,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,E92000001,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7.91
1333526,<NA>,<NA>,2018,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,E92000001,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,8.04
1333527,<NA>,<NA>,2019,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,E92000001,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,7.88


In [ ]:
#Unimos los dataset por el codigo y el anyo
dfs = [consumo_total, densidad.astype("string"), crimenes.astype("string"), p_vivienda.astype("string"), consumo_total]
consumo_total = reduce(lambda left,right: pd.merge(left,right, how="outer"), dfs)
consumo_total

KeyboardInterrupt: ignored

###ANALISIS DEL DATASET

##LIMPIEZA

##VALORES PERDIDOS

##OUTLIERS

##TRANSFORMACION

#MINERIA DE DATOS

#ALGORITMOS MACHINE LEARNING

#OPTIMZACION DE PARAMETROS